In [32]:
import pandas as pd
import numpy as np
from collections import Counter

In [11]:
df = pd.read_csv("../dataset/facebook_comment_tokenized_scored.tsv",sep='\t',encoding='utf-8')

In [12]:
df['scoredAmt'].value_counts()

0    151859
3      4895
1      3555
2      2308
Name: scoredAmt, dtype: int64

In [18]:
scored = df[df['scoredAmt']>0]
scored

Unnamed: 0  level_0   index  commenter_id  \
20              20       20      20  1.020198e+16   
25              25       25      25  7.288386e+14   
67              67       70      70  9.769091e+14   
106            106      110     110  7.499297e+14   
113            113      117     117  5.780900e+14   
116            116      121     121  9.731607e+14   
118            118      123     123  1.254079e+14   
122            122      127     127  1.113488e+15   
141            141      147     147  8.537187e+14   
160            160      168     168  7.512572e+14   
175            175      183     183  1.786072e+14   
203            203      211     211  1.177189e+15   
204            204      212     212  7.724293e+14   
232            232      241     241  6.448001e+14   
253            253      269     269  3.774301e+14   
255            255      271     271  1.746368e+15   
302            302      327     324  1.549523e+15   
305            305      330     327  1.578623e+15   
345            345      370     367  1.015288e+16   
349            349      374     371  3.676123e+14   
350            350      378     375  1.015235e+16   
352            352      382     379  1.879640e+14   
390            390      430     427  8.717984e+14   
392            392      432     429  7.059342e+14   
415            415      455     452  9.131607e+14   
426            426      466     463  8.011431e+14   
460            460      500     497  1.015261e+16   
472            472      513     510  1.879322e+14   
504            504      546     543  6.931296e+14   
537            537      579     576  1.148922e+15   
...            ...      ...     ...           ...   
162184      162184   172492  172487  5.749827e+14   
162202      162202   172510  172505  1.905100e+14   
162203      162203   172511  172506  3.907993e+14   
162234      162234   172543  172538  6.878547e+14   
162240      162240   172549  172544  7.012793e+14   
162267      162267   172576  172571  9.339007e+14   
162310      162310   172620  172615  8.967217e+14   
162311      162311   172621  172616  7.042389e+14   
162334      162334   172644  172639  3.908134e+14   
162339      162339   172649  172644  6.298433e+14   
162366      162366   172676  172671  7.264381e+14   
162391      162391   172701  172696  1.469388e+15   
162396      162396   172706  172701  3.917707e+14   
162401      162401   172711  172706  6.603298e+14   
162419      162419   172729  172724  4.667096e+14   
162430      162430   172740  172735  8.149281e+14   
162444      162444   172754  172749  6.556234e+14   
162477      162477   172787  172782  7.944434e+14   
162481      162481   172791  172786  3.762296e+14   
162485      162485   172795  172790  1.020318e+16   
162490      162490   172800  172795  6.410443e+14   
162497      162497   172807  172802  7.843962e+14   
162533      162533   172845  172840  4.415211e+14   
162538      162538   172850  172845  1.454323e+15   
162560      162560   172873  172868  2.005011e+14   
162568      162568   172881  172876  9.503352e+14   
162570      162570   172883  172878  3.383950e+14   
162585      162585   172898  172893  6.394843e+14   
162586      162586   172899  172894  1.827207e+15   
162603      162603   172916  172911  1.017020e+15   

                             commenter_name         created_time  like_count  \
20      ไม่รู้จะชื่ออะไรดี ใช้ไปก่อนแล้วกัน  2016-06-01 07:03:26         0.0   
25                           Ginnii Masawat  2016-06-03 03:14:40         0.0   
67                     เอกชัย แมททิว คงถาวร  2016-06-03 08:53:54         0.0   
106                           Nuthatai Nund  2016-06-04 15:49:26         1.0   
113                      Nattra Patrasiriya  2016-06-04 07:17:01         2.0   
116                              หลิว อลิษา  2016-06-05 04:49:06         0.0   
118                    นายฮาฮาฮา เอาฮาไปไหน  2016-06-04 07:12:00         0.0   
122                          Pranikorn Ipon  2016-06-04 13:23:22         2.0   
1

In [24]:
scored.to_csv('../dataset/facebook_comment_tokenized_scored.tsv',sep='\t',encoding='utf-8',index=False)

In [117]:
def computeJudgeScore(row):
    scores = [row['score_0'],row['score_1'],row['score_2']]

    if(row['scoredAmt']==1):
        return scores[0]
    elif(row['scoredAmt']==2):
        if(scores[0]==scores[1]): return scores[0]
        else: 
            if((scores[0]>0 and scores[1]>0) or (scores[0]<0 and scores[1]<0)):
                if(scores[0]==-32 or scores[1]==-32):
                    return "Drop_score: %s/%s"%(min(scores[0],scores[1]),max(scores[0],scores[1]))
                else:
                    return (scores[0]+scores[1])/2
            return "Drop_score: %s/%s"%(min(scores[0],scores[1]),max(scores[0],scores[1]))
    elif(row['scoredAmt']==3):
        counts = Counter(scores)
        if(counts.most_common()[0][1]>1): #Found majority
            return counts.most_common()[0][0]   
        else: #Unique. 
            scores2 = [score if score>-32 else 0 for score in scores] #change -32 to 0
            scores2.sort()
            if(scores2[0]<0 and scores2[1] < 0):#Negative Majority
                return -1.5
            elif(scores2[1]>0 and scores2[2]>0):
                return 1.5
            else:
                return "Drop_score: %s/%s/%s"%(scores[0],scores[1],scores[2])
            return "Drop_score: %s/%s/%s"%(scores[0],scores[1],scores[2])
    return "None"

scored['judgeScore'] = scored.apply(computeJudgeScore,axis=1)

In [125]:
scored['judgeScore'].value_counts()
select_scored = scored[scored['judgeScore'].apply(lambda x: "Drop_score" not in str(x))]

In [130]:
select_scored.to_csv("fb_comment_token_annotated.tsv",encoding='utf-8',index=False,sep='\t')

In [132]:
df = pd.read_csv("fb_comment_token_annotated.tsv",encoding='utf-8',sep='\t')

In [134]:
df['judgeScore'].value_counts()

-1.0     3046
-32.0    3018
 1.0     1493
 0.0     1222
-2.0      484
 2.0      274
-1.5      206
 1.5       90
Name: judgeScore, dtype: int64

In [135]:
df[df['judgeScore']==2]

commenter_id              commenter_name         created_time  \
0     7.288386e+14              Ginnii Masawat  2016-06-03 03:14:40   
21    8.011431e+14           Chaiwat Pratesang  2016-06-07 15:18:20   
25    1.148922e+15               Nisa SOo Good  2016-06-07 12:08:11   
90    8.011431e+14           Chaiwat Pratesang  2016-06-13 11:50:28   
148   1.548191e+14                     Sue Lay  2016-07-07 02:50:09   
234   6.036122e+14             ละมัยพร สารบรรณ  2016-06-24 13:20:07   
270   1.568337e+14               Frank Ohandza  2016-07-06 06:00:48   
316   8.305658e+14          Aummy Lucky Hoikom  2016-07-16 12:32:01   
325   1.172980e+14             ธนภูมิ พัฒบุญมา  2016-06-30 08:44:46   
350   9.266335e+14              Eldest Son Lee  2016-07-01 12:37:41   
375   1.069685e+15              Faii Arinthorn  2016-07-01 10:39:58   
377   7.027663e+14        Prapatsorn Kruamaung  2016-07-02 14:54:23   
395   7.490797e+14             Hanachungz Shop  2016-07-02 11:25:23   
397   8.273527e+14  เสี่ยเบศจัดหั้ย ตลอดน่ะจ๊ะ  2016-07-02 11:58:27   
411   1.566683e+15               Pep Nattaphon  2016-07-03 11:40:31   
415   1.748579e+15            Bunserm Panyadee  2016-07-05 00:34:32   
417   1.543170e+15                กาย มือศิลป์  2016-07-06 01:35:46   
420   1.069731e+14                Lek Resident  2016-07-08 11:48:53   
498   1.013090e+15             Lilly Carnation  2016-07-16 11:41:03   
504   1.013090e+15             Lilly Carnation  2016-07-16 11:41:08   
507   7.534705e+14             Phaarat Chairat  2016-07-16 07:32:01   
511   1.697503e+15               Jatsada Piala  2016-07-16 15:46:08   
513   1.477556e+14             ประหยัด ลาภเกิด  2016-09-24 07:18:26   
530   4.363985e+12              Takeshi Fighto  2016-07-20 19:27:12   
557   4.742503e+14     Patcharapong Pongpochai  2016-07-22 07:43:42   
575   1.928274e+14                 Koikoi Vivo  2016-07-29 09:41:17   
594   4.017505e+14          Koonkeag Udchachon  2016-07-25 13:18:21   
639   1.648376e+15        ชบาไพร ชบาไพร ชบาไพร  2016-08-10 05:04:57   
640   4.500075e+14          Kedwalai Boonjuang  2016-08-17 14:09:57   
656   1.395558e+15            Celebrities Idol  2016-08-02 08:36:43   
...            ...                         ...                  ...   
8462  1.422919e+15           สมจิต เลิศธนาภรณ์  2017-01-07 08:32:32   
8477  8.471928e+14              แมนคุง จอมยุ่ง  2017-01-12 15:44:27   
8488  7.305477e+14         น้ำเต้าหู้ แก่นมือง  2017-01-13 12:00:04   
8538  1.491192e+15                     ฅน แกดำ  2017-01-19 15:53:13   
8542  1.393219e+15    Ratthasat Rattanachumphu  2017-01-20 10:31:51   
8611  1.275494e+15                 Ton Abruzzi  2017-02-04 18:44:32   
8648  6.126454e+14                 ป้อม รามสอง  2017-02-07 11:03:46   
8701  3.830871e+14                สุนทร ขาวพรม  2017-02-18 02:18:49   
8715  1.446128e+15                   จรัส คำฟู  2017-02-21 04:30:16   
8774  3.969969e+14       มันคงจะดี ถ้าไม่มีกู'  2017-02-19 06:19:26   
8834  1.205119e+14       Wanyiwa Promduangsree  2017-02-24 06:23:18   
8956  4.409921e+14          Pisanuwat Gatewong  2017-03-09 17:38:09   
9086  7.946713e+14          Danuwat B. Bunhuad  2017-03-24 02:14:10   
9194  8.154823e+14                  Kathy Zhou  2017-03-28 05:08:28   
9195  4.080783e+14               Gam Pimchanok  2017-03-27 15:32:24   
9196  4.057783e+14                  น้อง จันจิ  2017-03-26 16:03:14   
9197  1.427301e+15                Tanadun Numa  2017-03-28 06:49:44   
9198  9.267746e+14        Chatchai Thiengkamol  2017-03-28 05:19:48   
9204  1.020366e+16              Au Patidtanont  2017-03-28 14:38:05   
9205  8.586805e+14     Pasinee Teerakantapirut  2017-03-31 15:22:44   
9207  1.015226e+16                Natt Machima  2017-03-31 06:32:42   
9225  2.803343e+14                       Na Ya  2017-03-26 08:16:21   
9327  7.520759e+14                Chaweewan Ph  2017-04-04 09:02:04   
9332  1.517346e+14                      มุ มิน  2017-04-04 08:44:27   
9353  1.020331e+16            

In [137]:
df.iloc[325]['message']

'วันนี้ไปแจ้งค่ายมาแล้วครับ เหลือไปจ่ายตังค่ายเดิม ก็ได้มาใช้ dtac แล้วครับ ขอบคุณสำหรับโปรดีๆน่ะครับ'